In this notebook, we collect as many links from the website of letsintern as we can. We use BeautifulSoup to scrape the data.

Also, note that the way links have been extracted is a lil weird or different if I may say. That is only because I am still not perfect in scraping and sometimes have to hack my way out.

We collect links from letsintern in the following way :
1. We go to its landing page and collect the links to all the internship categories.
2. We go to each of these categories and collect the 10-14 links that we can for particular internship profiles. More can't be collected because the page uses a button that has to be clicked to display more links. When the next section of links is displayed, the page stays the same w.r.t. its url. I haven't been able to figure out a way to deal with this as of now.
3. We go to to all the internship profiles and collect information from them. We also collect other internship profile links that have been linked to in that page(the ones being recommended on the side of the page). Note that these collected links point to compay pages where different internship profiles are listed. So after going to each of these links, internship profile links have to be extracted again.

This way we will get enough internship profiles to start our work with.

_Note: Complete links has been used below to refer to entire links. 
eg: www.letsintern.com/internships_

In [1]:
# importing libs
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [46]:
# defining functions
def collect_tag_links():
    '''
    collects all complete links to different intership category pages such as:
    internships in Delhi, Finance internships, Tech internships, internships in chandigarh etc.
    
    OUPUT:
    tag_links - returns all links to different present on www.letsintern.com 
    '''
    tag_links = []
    
    url = requests.get('https://www.letsintern.com/')
    data = url.text
    soup = BeautifulSoup(data)
    
    tags = soup.find_all('li',attrs = {'class':'col-sm-4'})
    for i in tags:
        hrefs = i.find_all('a')
        hrefs = ['http://letsintern.com' + j['href'] for j in hrefs]
        tag_links.extend(hrefs)
        
    return tag_links

def find_links_main(links_list):
    '''
    finds links to specific internship pages from pages where different internship profiles are listed. 
    Eg : www.letsintern.com/interships/IT-internships
    
    INPUT:
    links_list - a list of complete links that refer to different categorial pages
    
    OUPUT:
    collected_links - a list of collected links from all the pages in links_list 
    '''
    collected_links = []
    n = 1
    
    for link in links_list:
        url = requests.get(link)
        data = url.text
        soup = BeautifulSoup(data)
        
        links = soup.find_all('div', attrs = {'class':'job-title'})
        
        collected_links.extend(['http://letsintern.com' + i.a['href'] for i in links if 'letsintern' not in i.a['href']])
        
        print(n/len(links_list))
        n+= 1
    collected_links = list(set(collected_links))
    return collected_links

def find_links_internship(links_list):
    '''
    finds the links present on an internship profile page. These links point to a company 
    page where more links are posted. 
    Also, the links collected will be similar to the internship on that page as these themselves are the 
    recommendations by letsintern.
    
    INPUT:
    links_list - a list of complete links that refer to specific internship pages
    
    OUTPUT:
    collected_links - a list of collected links from all the pages in links_list after removing the 
                      links in the input links_list(ensuring we aren't returning duplicates).
                      
    '''
    collected_links = []
    n = 1
    
    for link in links_list:
        url = requests.get(link)
        data = url.text
        soup = BeautifulSoup(data)
        
        links = soup.find_all('div',attrs={'class':"col-sm-9 col-xs-9"})
        
        collected_links.extend(['http://letsintern.com' + i.a['href'] for i in links])
        
        print(n/len(links_list))
        n+= 1
    collected_links = list(set(collected_links))
    return collected_links

def find_links_company(links_list):
    '''
    finds the links to different internship profiles present on an internship company page. 
    
    INPUT:
    links_list - a list of complete links that refer to specific company pages
    
    OUTPUT:
    collected_links - a list of collected links from all the pages in links_list after removing the 
                      links in links_list.
    '''
    collected_links = []
    n = 1
                           
    for link in links_list:
        url = requests.get(link)
        data = url.text
        soup = BeautifulSoup(data)
                           
        links = soup.find_all('div',attrs = {'class':'job-title'})
                           
        for i in links:
            try:
                collected_links.append('http://letsintern.com'+ i.a['href'])
            except:
                print(i.a)
                continue
        print(n/len(links_list))
        n+= 1
    return list(set(collected_links))

def extract_data(links_list):
    '''
    extracts all the relevant data needed from each of the links and returns and saves a 
    dataframe containing all that information.
    
    INPUT:
    links_list - a list of complete links that refer to specific internship pages
    
    OUTPUT:
    df - a dataframe with the rows as the links and the columns as the information extracted
    
    '''
    job_title = []
    company_name = []
    job_loc = []
    details = []
    category = []
    compensation =[]
    start = []
    end = []
    skills = []
    hrefs = []
    n = 1
    
    for link in links_list:
        url = requests.get(link)
        data = url.text
        soup = BeautifulSoup(data) 

        try :
            # many job titles were not given as the pages didn't exist themselves
            job_title.append(soup.find_all('div',attrs={'class':'job-title'})[0].text)
        except:
            print(link)
            links_list.remove(link)
            continue # continue breaks the current iteration of the loop and jumps to the next one
        try:                 
            company_name.append(soup.find_all('div', attrs ={'class':'company-name'})[0].text)
        except:
            print(soup.find_all('div', attrs ={'class':'company-name'}))
            company_name.append('no company found')
        try:
            job_loc.append(soup.find_all('div', attrs ={'class':'job-locations'})[0].text)
        except:
            print(soup.find_all('div', attrs ={'class':'job-locations'}))
            job_loc.append('no job location found')
        try:
            details.append(soup.find_all('div', attrs ={'class':'details-section fixht'})[0].text)
        except:
            print(soup.find_all('div', attrs ={'class':'details-section fixht'}))
            details.append('no details found')
        try:  
            category.append(soup.find_all('a', attrs= {'title':'Internship Category'})[0].text)
        except:
            print(soup.find_all('a', attrs= {'title':'Internship Category'}))
            category.append('no category found')
        try:
            compensation.append(soup.find_all('a', attrs= {'title':'Compensation Type'})[0].text)
        except: 
            print(soup.find_all('a', attrs= {'title':'Compensation Type'}))
            compensation.append('no compensation found')
        try:
            start.append(soup.find_all('li', attrs = {'title':'Start Date'})[0].text)
        except:
            print(soup.find_all('li', attrs = {'title':'Start Date'}))
            start.append('no start date found')
        try:    
            end.append(soup.find_all('li', attrs = {'title':'End Date'})[0].text)
        except:
            print(soup.find_all('li', attrs = {'title':'End Date'}))
            end.append('no end date found')
        try:
            skills.append(soup.find_all('div', attrs = {'id':'skills-required'})[0].text)
        except: 
            print(soup.find_all('div', attrs = {'id':'skills-required'}))
            skills.append('no skills found')
        hrefs.append(link)
        print(n/len(links_list))
        n+=1
       

    df = pd.DataFrame({'href':hrefs, 'job_title':job_title, 'company_name':company_name, 'job_loc':job_loc
                      ,'details':details, 'category':category, 'compensation':compensation, 'start':start
                      ,'end':end, 'skills':skills})
    df.to_csv('../data/information_from_links.csv')
    return df


#### Our approach to this would be as follows:
1. Call `collect_tag_links()` to collect all the main internship listing pages.
2. Call `find_links_main()` on all of these. Save the collected links separately.
3. Call `find_links_internships()` on all of the links collected in the above step. Save these links separately.
4. Call `find_links_company()` on the above links.
5. Take the `list(set())` of the links in 2 and 4 step.

These steps will give us all the links we need to scrape information from

In [3]:
tag_links = collect_tag_links()

In [4]:
saved_links = find_links_main(tag_links)

0.03571428571428571
0.07142857142857142
0.10714285714285714
0.14285714285714285
0.17857142857142858
0.21428571428571427
0.25
0.2857142857142857
0.32142857142857145
0.35714285714285715
0.39285714285714285
0.42857142857142855
0.4642857142857143
0.5
0.5357142857142857
0.5714285714285714
0.6071428571428571
0.6428571428571429
0.6785714285714286
0.7142857142857143
0.75
0.7857142857142857
0.8214285714285714
0.8571428571428571
0.8928571428571429
0.9285714285714286
0.9642857142857143
1.0


In [5]:
saved_links_1 = find_links_internship(saved_links)

0.006369426751592357
0.012738853503184714
0.01910828025477707
0.025477707006369428
0.03184713375796178
0.03821656050955414
0.044585987261146494
0.050955414012738856
0.05732484076433121
0.06369426751592357
0.07006369426751592
0.07643312101910828
0.08280254777070063
0.08917197452229299
0.09554140127388536
0.10191082802547771
0.10828025477707007
0.11464968152866242
0.12101910828025478
0.12738853503184713
0.1337579617834395
0.14012738853503184
0.1464968152866242
0.15286624203821655
0.1592356687898089
0.16560509554140126
0.17197452229299362
0.17834394904458598
0.18471337579617833
0.1910828025477707
0.19745222929936307
0.20382165605095542
0.21019108280254778
0.21656050955414013
0.2229299363057325
0.22929936305732485
0.2356687898089172
0.24203821656050956
0.2484076433121019
0.25477707006369427
0.2611464968152866
0.267515923566879
0.27388535031847133
0.2802547770700637
0.28662420382165604
0.2929936305732484
0.29936305732484075
0.3057324840764331
0.31210191082802546
0.3184713375796178
0.3248407

In [6]:
saved_links_2 = find_links_company(saved_links_1)

0.0028089887640449437
0.0056179775280898875
0.008426966292134831
0.011235955056179775
0.014044943820224719
0.016853932584269662
0.019662921348314606
0.02247191011235955
0.025280898876404494
0.028089887640449437
0.03089887640449438
0.033707865168539325
0.03651685393258427
0.03932584269662921
0.042134831460674156
0.0449438202247191
0.047752808988764044
0.05056179775280899
0.05337078651685393
0.056179775280898875
0.05898876404494382
0.06179775280898876
0.06460674157303371
0.06741573033707865
0.0702247191011236
0.07303370786516854
0.07584269662921349
0.07865168539325842
<a class="blank-link" data-ecategory="job_title" data-etracking="true" data-let="bro" target="_blank" title="Embedded development / Application Development / ERP/System testing">Embedded development / Application Development / ERP/System testing</a>
0.08146067415730338
0.08426966292134831
0.08707865168539326
0.0898876404494382
0.09269662921348315
0.09550561797752809
0.09831460674157304
0.10112359550561797
0.1039325842696629

0.9943820224719101
0.9971910112359551
1.0


In [8]:
final_links = list(set(saved_links_2) | set(saved_links))

In [9]:
len(final_links)

667

We had 665 unique links when I tried it earlier. Thats the data I have worked on.

_Note: this number can change depending on when you run it because the website keeps getting updated with new internships_

In [47]:
# extract the data from all the links
df = extract_data(final_links)

0.0015337423312883436
0.003067484662576687
0.004601226993865031
0.006134969325153374
0.007668711656441718
0.009202453987730062
0.010736196319018405
0.012269938650306749
0.013803680981595092
0.015337423312883436
0.01687116564417178
0.018404907975460124
0.019938650306748466
0.02147239263803681
0.023006134969325152
0.024539877300613498
0.02607361963190184
0.027607361963190184
0.029141104294478526
0.03067484662576687
0.032208588957055216
0.03374233128834356
0.0352760736196319
0.03680981595092025
0.03834355828220859
0.03987730061349693
0.04141104294478527
0.04294478527607362
0.04447852760736196
0.046012269938650305
0.04754601226993865
0.049079754601226995
0.05061349693251534
0.05214723926380368
0.05368098159509203
0.05521472392638037
0.05674846625766871
0.05828220858895705
0.0598159509202454
0.06134969325153374
0.06288343558282208
0.06441717791411043
0.06595092024539877
0.06748466257668712
0.06901840490797546
0.0705521472392638
0.07208588957055215
0.0736196319018405
0.07515337423312883
0.07

0.6487730061349694
0.6503067484662577
0.651840490797546
0.6533742331288344
0.6549079754601227
0.656441717791411
0.6579754601226994
0.6595092024539877
0.661042944785276
0.6625766871165644
0.6641104294478528
0.6656441717791411
0.6671779141104295
0.6687116564417178
0.6702453987730062
0.6717791411042945
0.6733128834355828
0.6748466257668712
0.6763803680981595
0.6779141104294478
0.6794478527607362
0.6809815950920245
0.6825153374233128
0.6840490797546013
0.6855828220858896
0.6871165644171779
0.6886503067484663
0.6901840490797546
0.691717791411043
0.6932515337423313
0.6947852760736196
0.696319018404908
0.6978527607361963
0.6993865030674846
0.700920245398773
0.7024539877300614
0.7039877300613497
0.7055214723926381
0.7070552147239264
0.7085889570552147
0.7101226993865031
0.7116564417177914
0.7131901840490797
0.7147239263803681
0.7162576687116564
0.7177914110429447
0.7193251533742331
0.7208588957055214
0.7223926380368099
0.7239263803680982
0.7254601226993865
0.7269938650306749
0.7285276073619632